In [16]:
from main import agent_executor

In [18]:
import os
import requests
from dotenv import load_dotenv

load_dotenv()

env_url = os.getenv("INCORTA_ENV_URL")
tenant = os.getenv("INCORTA_TENANT")
user = os.getenv("INCORTA_USERNAME")
password = os.getenv("INCORTA_PASSWORD")

def login_with_credentials(env_url, tenant, user, password):
    """
    Logs into Incorta using username, password, and tenant credentials.
    Returns a session with relevant authentication details.
    """
    response = requests.post(
        f"{env_url}/authservice/login",
        data={"tenant": tenant, "user": user, "pass": password},
        verify=True,
        timeout=60
    )

    if response.status_code != 200:
        response.raise_for_status()

        # Extract session cookies
    id_cookie, login_id = None, None
    for item in response.cookies.items():
        if item[0].startswith("JSESSIONID"):
            id_cookie, login_id = item
            break

    if not id_cookie or not login_id:
        raise Exception("Failed to retrieve session cookies during login.")

        # Verify login and retrieve CSRF token
    response = requests.get(
        f"{env_url}/service/user/isLoggedIn",
        cookies={id_cookie: login_id},
        verify=True,
        timeout=60
    )

    if response.status_code != 200 or "XSRF-TOKEN" not in response.cookies:
        raise Exception(f"Failed to log in to {env_url} for tenant {tenant} using user {user}. Please verify credentials.")

        # Retrieve CSRF token and access token
    csrf_token = response.cookies["XSRF-TOKEN"]
    authorization = response.json().get("accessToken")

    if not authorization:
        raise Exception("Failed to retrieve access token during login.")

    return {
        "env_url": env_url,
        "id_cookie": id_cookie,
        "id": login_id,
        "csrf": csrf_token,
        "authorization": authorization,
        "verify": True,
        "session_cookie": {id_cookie: login_id, "XSRF-TOKEN": csrf_token}
    }


def fetch_schema_details(schema_name: str):
    """Fetch schema details from the Incorta environment."""

    login_creds = login_with_credentials(env_url, tenant, user, password)

    url = f"{env_url}/bff/v1/schemas/name/{schema_name}"

    cookie = ""
    for key, value in login_creds['session_cookie'].items():
        cookie += f"{key}={value};"

    headers = {
        "Authorization": f"Bearer {login_creds['authorization']}",
        "Content-Type": "application/json",
        "X-XSRF-TOKEN": login_creds["csrf"],
        "Cookie": cookie
    }

    response = requests.get(url, headers=headers, verify=False)

    if response.status_code == 200:
        response_data = response.json()

        return {"schema_details": response_data}
    else:
        return {"error": f"Failed to fetch schema details: {response.status_code} - {response.text}"}

In [19]:
fetch_schema_details("ZendeskTickets")

/Users/habiba/Documents/Internal PM Tool/Source Code/.venv/lib/python3.9/site-packages/urllib3/connectionpool.py:1097: InsecureRequestWarning: Unverified HTTPS request is being made to host 'my-incorta.cloud.incorta.com'. Adding certificate verification is strongly advised. See: https://urllib3.readthedocs.io/en/latest/advanced-usage.html#tls-warnings
  warnings.warn(


{'schema_details': {'name': 'ZendeskTickets',
  'schemaId': 273,
  'ownerId': 883,
  'ownerName': 'My Incorta',
  'permissionCode': '0',
  'tables': [{'name': 'ticket_comments',
    'type': 'view',
    'alias': False,
    'multiDataSource': False,
    'cached': True,
    'snapshot': False,
    'enablePkIndexOnFullLoad': True,
    'directoryBased': False,
    'maxColumnNumber': 8,
    'connectorId': -1,
    'loadOrder': 1,
    'partialLoading': 'none',
    'dataRetention': 'none',
    'deletionDataSetEnabled': False,
    'cascadePurge': False,
    'columns': [{'name': 'id',
      'label': 'Id',
      'function': 'key',
      'hide': False,
      'encrypt': False,
      'autoDetectType': True,
      'isPrimaryKey': False,
      'isForeignKey': False,
      'index': 1,
      'physicalName': 'id',
      'length': 0,
      'dateFormat': '',
      'mappedDataType': 'long'},
     {'name': 'ticket_id',
      'label': 'Ticket Id',
      'function': 'dimension',
      'hide': False,
      'encry

In [21]:
fetch_schema_details("Community")

/Users/habiba/Documents/Internal PM Tool/Source Code/.venv/lib/python3.9/site-packages/urllib3/connectionpool.py:1097: InsecureRequestWarning: Unverified HTTPS request is being made to host 'my-incorta.cloud.incorta.com'. Adding certificate verification is strongly advised. See: https://urllib3.readthedocs.io/en/latest/advanced-usage.html#tls-warnings
  warnings.warn(


{'schema_details': {'name': 'Community',
  'schemaId': 1129,
  'description': 'Incorta Forumbee Community',
  'ownerId': 7760,
  'ownerName': 'Joseph Miller',
  'permissionCode': '0',
  'tables': [{'name': 'Community_Bulk_Data',
    'type': 'view',
    'alias': False,
    'multiDataSource': False,
    'cached': True,
    'snapshot': False,
    'enablePkIndexOnFullLoad': False,
    'directoryBased': False,
    'maxColumnNumber': -1,
    'connectorId': -1,
    'loadOrder': 1,
    'partialLoading': 'none',
    'dataRetention': 'none',
    'deletionDataSetEnabled': False,
    'cascadePurge': False,
    'columns': [{'name': 'RegistrationStatus',
      'label': 'Registration Status',
      'formula': 'case(Community.Community_Bulk_Data.user_registration_status="FULLY_REGISTERED", "Registered","Anonymous")',
      'function': 'dimension',
      'hide': False,
      'encrypt': False,
      'autoDetectType': True,
      'isPrimaryKey': False,
      'isForeignKey': False,
      'length': 0,
    